# Clean Up and Data Enrichment 

In [1]:
#We import all the modules that we will use in for the clean up and enrichment
import requests 
import json
import os as os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
import seaborn as sns
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import functions as fu
import Download_data as dd

In [2]:
dd.download_data_sets()


introduce the link to kaggle dataset: https://www.kaggle.com/sudalairajkumar/cryptocurrencypricehistory


0

In [3]:
!ls


An Analysis of 5 years of asset prices.ipynb
Clean Up and Encrichment.ipynb
Code For Cleanup and Visualization.ipynb
Download_data.py
ETHERUM & STOCK PRICES.ipynb
Predicting Stock Price.ipynb
Project V.1.ipynb
__pycache__
archivos.txt
database
functions.py
my-project.txt


## Cleaning the Crypto Dataframe

In [2]:
#We extract the csv files that we are interested in analyzing, in this case it will be Etherum and Bitcoin
df_bitcoin = pd.read_csv('database/coin_Bitcoin.csv', parse_dates = ['Date'])
df_ether = pd.read_csv('database/coin_Ethereum.csv', parse_dates = ['Date'])

In [3]:
#We turn the csv's into a dataframes.
ether = pd.DataFrame(df_ether)
bitcoin = pd.DataFrame(df_bitcoin)


In [4]:
#We notice that the Date column is in datetime, we are interested in only getting the date for our analysis. 
ether.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Ethereum,ETH,2015-08-08 23:59:59,2.798810,0.714725,2.793760,0.753325,674188.0,4.548689e+07
1,2,Ethereum,ETH,2015-08-09 23:59:59,0.879810,0.629191,0.706136,0.701897,532170.0,4.239957e+07
2,3,Ethereum,ETH,2015-08-10 23:59:59,0.729854,0.636546,0.713989,0.708448,405283.0,4.281836e+07
3,4,Ethereum,ETH,2015-08-11 23:59:59,1.131410,0.663235,0.708087,1.067860,1463100.0,6.456929e+07
4,5,Ethereum,ETH,2015-08-12 23:59:59,1.289940,0.883608,1.058750,1.217440,2150620.0,7.364501e+07


In [5]:
#We create a new date column that only includes the day, and drop the Date column we don't need anymore. 
#We also drop other columns that we will not use any more. 
ether['date'] = ether['Date'].dt.date
bitcoin['date'] = bitcoin['Date'].dt.date
bitcoin.drop(['Marketcap', 'Open', 'SNo', 'Name', 'Date'], axis =1, inplace = True)
ether.drop([ 'Marketcap', 'Open', 'SNo', 'Name', 'Date'], axis =1, inplace = True)


In [6]:
ether.head()

,Symbol,High,Low,Close,Volume,date
0,ETH,2.798810,0.714725,0.753325,674188.0,2015-08-08
1,ETH,0.879810,0.629191,0.701897,532170.0,2015-08-09
2,ETH,0.729854,0.636546,0.708448,405283.0,2015-08-10
3,ETH,1.131410,0.663235,1.067860,1463100.0,2015-08-11
4,ETH,1.289940,0.883608,1.217440,2150620.0,2015-08-12


In [7]:
#We apply our Return function and our previous close function to our crypto dataframes. 

ether = fu.crypto_prev_close(ether)
ether = fu.crypto_return(ether)
bitcoin = fu.crypto_prev_close(bitcoin)
bitcoin = fu.crypto_return(bitcoin)

In [8]:
ether.head()

,Symbol,High,Low,Close,Volume,date,previousClose,percent_change
0,ETH,2.798810,0.714725,0.753325,674188.0,2015-08-08,NaN,NaN
1,ETH,0.879810,0.629191,0.701897,532170.0,2015-08-09,0.753325,-0.068268
2,ETH,0.729854,0.636546,0.708448,405283.0,2015-08-10,0.701897,0.009333
3,ETH,1.131410,0.663235,1.067860,1463100.0,2015-08-11,0.708448,0.507323
4,ETH,1.289940,0.883608,1.217440,2150620.0,2015-08-12,1.067860,0.140075


In [9]:
#We create a df that combines both our bitcoin and etherum values
bit_eth2 = bitcoin.merge(ether, how = 'left', on = 'date', suffixes = ('_bit', '_eth') )
bit_eth2

,Symbol_bit,High_bit,Low_bit,Close_bit,Volume_bit,date,previousClose_bit,percent_change_bit,Symbol_eth,High_eth,Low_eth,Close_eth,Volume_eth,previousClose_eth,percent_change_eth
0,BTC,147.488007,134.000000,144.539993,0.000000e+00,2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BTC,146.929993,134.050003,139.000000,0.000000e+00,2013-04-30,144.539993,-0.038328,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BTC,139.889999,107.720001,116.989998,0.000000e+00,2013-05-01,139.000000,-0.158345,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BTC,125.599998,92.281898,105.209999,0.000000e+00,2013-05-02,116.989998,-0.100692,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BTC,108.127998,79.099998,97.750000,0.000000e+00,2013-05-03,105.209999,-0.070906,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,BTC,54204.929756,45290.590268,48824.426869,1.061025e+11,2021-02-23,54207.319065,-0.099302,ETH,1781.409018,1378.840907,1570.203925,5.202986e+10,1781.992976,-0.118850
2858,BTC,51290.136695,47213.498162,49705.333316,6.369552e+10,2021-02-24,48824.426869,0.018042,ETH,1710.983795,1511.018963,1626.575705,3.132900e+10,1570.203925,0.035901
2859,BTC,51948.966982,47093.853019,47093.853019,5.450657e+10,2021-02-25,49705.333316,-0.052539,ETH,1670.224062,1465.058946,1475.703735,2.448168e+10,1626.575705,-0.092754
2860,BTC,48370.785260,44454.842114,46339.760083,3.509679e+11,2021-02-26,47093.853019,-0.016013,ETH,1559.028891,1407.979253,1446.033650,3.143600e+10,1475.703735,-0.020106


In [10]:
#We grab the values for the age range we are interested in for our analysis 
#We will start our analysis from april 11 2016, as for the stock data we will use later that is the furthest date we will analyze.
startdate = pd.to_datetime("2016-04-11").date()
bit_eth = bit_eth2.loc[bit_eth2.loc[:, "date"] >= startdate]

In [11]:
bit_eth

,Symbol_bit,High_bit,Low_bit,Close_bit,Volume_bit,date,previousClose_bit,percent_change_bit,Symbol_eth,High_eth,Low_eth,Close_eth,Volume_eth,previousClose_eth,percent_change_eth
1078,BTC,422.739014,420.532990,422.483002,5.074750e+07,2016-04-11,421.563995,0.002180,ETH,8.946640,8.003180,8.643790,1.513810e+07,8.936340,-0.032737
1079,BTC,427.277008,422.842987,425.190002,7.072880e+07,2016-04-12,422.483002,0.006407,ETH,8.639470,7.154160,7.442310,2.319010e+07,8.643790,-0.138999
1080,BTC,426.657990,422.915985,423.734009,6.906040e+07,2016-04-13,425.190002,-0.003424,ETH,8.719770,7.069220,8.044420,3.140330e+07,7.442310,0.080904
1081,BTC,425.371002,423.013000,424.282013,4.528100e+07,2016-04-14,423.734009,0.001293,ETH,8.757040,8.021520,8.387240,1.359070e+07,8.044420,0.042616
1082,BTC,429.928009,424.427002,429.713013,5.480150e+07,2016-04-15,424.282013,0.012800,ETH,8.616200,8.097330,8.242780,1.217900e+07,8.387240,-0.017224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,BTC,54204.929756,45290.590268,48824.426869,1.061025e+11,2021-02-23,54207.319065,-0.099302,ETH,1781.409018,1378.840907,1570.203925,5.202986e+10,1781.992976,-0.118850
2858,BTC,51290.136695,47213.498162,49705.333316,6.369552e+10,2021-02-24,48824.426869,0.018042,ETH,1710.983795,1511.018963,1626.575705,3.132900e+10,1570.203925,0.035901
2859,BTC,51948.966982,47093.853019,47093.853019,5.450657e+10,2021-02-25,49705.333316,-0.052539,ETH,1670.224062,1465.058946,1475.703735,2.448168e+10,1626.575705,-0.092754
2860,BTC,48370.785260,44454.842114,46339.760083,3.509679e+11,2021-02-26,47093.853019,-0.016013,ETH,1559.028891,1407.979253,1446.033650,3.143600e+10,1475.703735,-0.020106


In [12]:
#We create a subdataframe with the return for both Cryptocoins
return_crypto = bit_eth[['percent_change_bit', 'percent_change_eth', 'date']]

#We set the index to datetime same as stock market data
return_crypto = return_crypto.set_index('date')

return_crypto.to_csv('../data/crypto_return.csv')

In [22]:
return_crypto

,percent_change_bit,percent_change_eth
date,,
2016-04-11,0.002180,-0.032737
2016-04-12,0.006407,-0.138999
2016-04-13,-0.003424,0.080904
2016-04-14,0.001293,0.042616
2016-04-15,0.012800,-0.017224
...,...,...
2021-02-23,-0.099302,-0.118850
2021-02-24,0.018042,0.035901
2021-02-25,-0.052539,-0.092754


# Extracting Stock Information With The Yahoo Finance API

In [13]:
url_yahoo = "https://yahoo-finance-low-latency.p.rapidapi.com/v8/finance/spark"
parameters = {"symbols":"AAPL,MSFT,GME,^GSPC,","range":"5y","interval":"1d"}
headers = {
    'x-rapidapi-key': "5978a55496msh35be22b262fe8acp19f68cjsn3abe53806b27",
    'x-rapidapi-host': "yahoo-finance-low-latency.p.rapidapi.com"
    }
response = requests.request("GET", url_yahoo, headers=headers, params=parameters)


In [14]:
#We create separate dataframes for our 4 stocks
response1 = response.json()
MSFT = pd.DataFrame(response1['MSFT'])
AAPL = pd.DataFrame(response1['AAPL'])
GME = pd.DataFrame(response1['GME'])
GSPC = pd.DataFrame(response1['^GSPC'])

In [16]:
#We change the timestamp column to a date time column using our predefined function
msft = fu.ts_to_dt(MSFT, 'timestamp', 'date')
aapl = fu.ts_to_dt(AAPL, 'timestamp', 'date')
gme = fu.ts_to_dt(GME, 'timestamp', 'date')
gspc = fu.ts_to_dt(GSPC, 'timestamp', 'date')

In [17]:
msft

,symbol,end,start,timestamp,dataGranularity,close,previousClose,chartPreviousClose,date
0,MSFT,None,None,1460381400,300,54.31,None,54.42,2016-04-11 14:30:00
1,MSFT,None,None,1460467800,300,54.65,None,54.42,2016-04-12 14:30:00
2,MSFT,None,None,1460554200,300,55.35,None,54.42,2016-04-13 14:30:00
3,MSFT,None,None,1460640600,300,55.36,None,54.42,2016-04-14 14:30:00
4,MSFT,None,None,1460727000,300,55.65,None,54.42,2016-04-15 14:30:00
...,...,...,...,...,...,...,...,...,...
1254,MSFT,None,None,1617629400,300,249.07,None,54.42,2021-04-05 14:30:00
1255,MSFT,None,None,1617715800,300,247.86,None,54.42,2021-04-06 14:30:00
1256,MSFT,None,None,1617802200,300,249.90,None,54.42,2021-04-07 14:30:00
1257,MSFT,None,None,1617888600,300,253.25,None,54.42,2021-04-08 14:30:00


In [19]:
#We apply the previous_close function to all the stock dataframes, 
#since our API scrapped data doesnt contain the right values for that column
SP500 = fu.previous_close(gspc)
AAPL = fu.previous_close(aapl)
MSFT = fu.previous_close(msft)
GME = fu.previous_close(gme)

In [20]:
#drop unneed columns for 4 stocks data frames
AAPL.drop(['end', 'start', 'timestamp', 'dataGranularity', 'chartPreviousClose'], axis =1, inplace = True)
SP500.drop(['end', 'start', 'timestamp', 'dataGranularity', 'chartPreviousClose'], axis =1, inplace = True)
GME.drop(['end', 'start', 'timestamp', 'dataGranularity', 'chartPreviousClose'], axis =1, inplace = True)
MSFT.drop(['end', 'start', 'timestamp', 'dataGranularity','chartPreviousClose'], axis =1, inplace = True)

In [23]:
#We extract only the date from the date column 
MSFT['date'] = MSFT['date'].dt.date
GME['date'] = GME['date'].dt.date
AAPL['date'] = AAPL['date'].dt.date
SP500['date'] = SP500['date'].dt.date

In [24]:
#We apply percent_change function to create new % change column
aapl = fu.percent_change(AAPL)
sp500 = fu.percent_change(SP500)
msft = fu.percent_change(MSFT)
gme = fu.percent_change(GME)

In [25]:
aapl

,symbol,close,previousClose,date,percent_change
0,AAPL,27.255,None,2016-04-11,NaN
1,AAPL,27.610,27.255,2016-04-12,0.013025
2,AAPL,28.010,27.61,2016-04-13,0.014488
3,AAPL,28.025,28.01,2016-04-14,0.000536
4,AAPL,27.462,28.025,2016-04-15,-0.020089
...,...,...,...,...,...
1254,AAPL,125.900,123,2021-04-05,0.023577
1255,AAPL,126.210,125.9,2021-04-06,0.002462
1256,AAPL,127.900,126.21,2021-04-07,0.013390
1257,AAPL,130.360,127.9,2021-04-08,0.019234


## Price Index Time Visualization Data Cleaning

In [26]:
#We create a price index column for all stock dataframes. 
#This is a way to understand the data better by creating a price column that follows changes to the first reported price.
gme2 = fu.base_price(gme)
aapl2 = fu.base_price(aapl)
msft2 = fu.base_price(msft)
sp501 = fu.base_price(sp500)

In [28]:
#Set the index as the date column for our price index stocks
aapl_pi = aapl2.set_index('date')
msft_pi = msft2.set_index('date')
gme_pi = gme2.set_index('date')
sp500_pi = sp501.set_index('date')

In [30]:
aapl_pi

,symbol,close,previousClose,percent_change,price_index
date,,,,,
2016-04-11,AAPL,27.255,None,NaN,1.000000
2016-04-12,AAPL,27.610,27.255,0.013025,1.013025
2016-04-13,AAPL,28.010,27.61,0.014488,1.027701
2016-04-14,AAPL,28.025,28.01,0.000536,1.028252
2016-04-15,AAPL,27.462,28.025,-0.020089,1.007595
...,...,...,...,...,...
2021-04-05,AAPL,125.900,123,0.023577,4.619336
2021-04-06,AAPL,126.210,125.9,0.002462,4.630710
2021-04-07,AAPL,127.900,126.21,0.013390,4.692717


In [31]:
# We merge all the price index into one data frame 
ms_aap_pi = msft_pi.merge(aapl_pi, how = 'left', on = 'date', suffixes = ('_msft', '_aapl') )
ms_aap_gme_pi = ms_aap_pi.merge(gme_pi, how = 'left', on = 'date', suffixes = ('_msft', '_aapl', '_gme'))
all_tog_pi = ms_aap_gme_pi.merge(sp500_pi, how = 'left', on = 'date', suffixes = ('_gme', '_sp500'))

In [35]:
#We create a subdataframe with the information we are interested in to calculate a time series of the base price
price_index = all_tog_pi[['price_index_msft', 'price_index_aapl', 'price_index_gme', 'price_index_sp500']]
price_index.to_csv('../data/stocks_price_index.csv')

In [36]:
#We Keep only the values of Ether that we are interested in.
startdate = pd.to_datetime("2016-04-11").date()
ether = ether.loc[ether.loc[:, 'date'] >= startdate]
bitcoin = bitcoin.loc[bitcoin.loc[:, 'date'] >= startdate]
ether = ether.reset_index()
bitcoin = bitcoin.reset_index()

In [40]:
#We create the bitcoin base price index column, and extract the data frames to csv
ether_indexed = fu.crypto_index(ether)
bit_indexed = fu.crypto_index(bitcoin)
bit_indexed.to_csv('../data/bitcoin_price_index.csv')
ether_indexed.to_csv('../data/etherum_price_index.csv')

In [44]:
# We merge the crypto indexed dataframes to one dataframe
crypto_indexed = bit_indexed.merge(ether_indexed, on = 'date', how = 'left', suffixes = ('_bit', '_ether'))
crypto_indexed = crypto_indexed[['price_index_bit', 'price_index_ether', 'date']]

crypto_indexed.set_index('date', inplace = True)

crypto_indexed.to_csv('../data/base_price_crypto.csv')

In [45]:
crypto_indexed

,price_index_bit,price_index_ether
date,,
2016-04-11,1.000000,1.000000
2016-04-12,1.006407,0.861001
2016-04-13,1.002961,0.930659
2016-04-14,1.004258,0.970320
2016-04-15,1.017113,0.953607
...,...,...
2021-02-23,115.565423,181.656875
2021-02-24,117.650493,188.178526
2021-02-25,111.469226,170.724149


# Cleaning Data for the Sharpe Ratio

In [57]:
#Set the index as the date column for our stocks
msft = msft.set_index('date')
aapl = aapl.set_index('date')
sp500 = sp500.set_index('date')
gme = gme.set_index('date')


KeyError: "None of ['date'] are in the columns"

In [58]:
msft

,symbol,close,previousClose,percent_change,price_index
date,,,,,
2016-04-11,MSFT,54.31,None,NaN,1.000000
2016-04-12,MSFT,54.65,54.31,0.006260,1.006260
2016-04-13,MSFT,55.35,54.65,0.012809,1.019149
2016-04-14,MSFT,55.36,55.35,0.000181,1.019333
2016-04-15,MSFT,55.65,55.36,0.005238,1.024673
...,...,...,...,...,...
2021-04-05,MSFT,249.07,242.35,0.027728,4.586080
2021-04-06,MSFT,247.86,249.07,-0.004858,4.563800
2021-04-07,MSFT,249.90,247.86,0.008230,4.601363


In [50]:
#Creating a Df of returns for our 2 stocks so that we can subtract the sp_500 returns
#this is done to calculate the sharpe ratio
ms_aap = msft.merge(aapl, how = 'left', on = 'date', suffixes = ('_msft', '_aapl') )

all_together = ms_aap.merge(sp500, how = 'left', on ='date', suffixes = ('_msft', '_aapl', '_sp500'))

all_together2 = all_together.merge(gme, how = 'left', on = 'date', suffixes = ('_sp500', '_gme'))

all_together2

,symbol_msft,close_msft,previousClose_msft,percent_change_msft,price_index_msft,symbol_aapl,close_aapl,previousClose_aapl,percent_change_aapl,price_index_aapl,symbol_sp500,close_sp500,previousClose_sp500,percent_change_sp500,price_index_sp500,symbol_gme,close_gme,previousClose_gme,percent_change_gme,price_index_gme
date,,,,,,,,,,,,,,,,,,,,
2016-04-11,MSFT,54.31,None,NaN,1.000000,AAPL,27.255,None,NaN,1.000000,^GSPC,2041.99,None,NaN,1.000000,GME,29.88,None,NaN,1.000000
2016-04-12,MSFT,54.65,54.31,0.006260,1.006260,AAPL,27.610,27.255,0.013025,1.013025,^GSPC,2061.72,2041.99,0.009662,1.009662,GME,30.09,29.88,0.007028,1.007028
2016-04-13,MSFT,55.35,54.65,0.012809,1.019149,AAPL,28.010,27.61,0.014488,1.027701,^GSPC,2082.42,2061.72,0.010040,1.019799,GME,31.47,30.09,0.045862,1.053213
2016-04-14,MSFT,55.36,55.35,0.000181,1.019333,AAPL,28.025,28.01,0.000536,1.028252,^GSPC,2082.78,2082.42,0.000173,1.019976,GME,31.65,31.47,0.005720,1.059237
2016-04-15,MSFT,55.65,55.36,0.005238,1.024673,AAPL,27.462,28.025,-0.020089,1.007595,^GSPC,2080.73,2082.78,-0.000984,1.018972,GME,31.79,31.65,0.004423,1.063922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-05,MSFT,249.07,242.35,0.027728,4.586080,AAPL,125.900,123,0.023577,4.619336,^GSPC,4077.91,4019.87,0.014438,1.997027,GME,186.95,191.45,-0.023505,6.256693
2021-04-06,MSFT,247.86,249.07,-0.004858,4.563800,AAPL,126.210,125.9,0.002462,4.630710,^GSPC,4073.94,4077.91,-0.000974,1.995083,GME,184.50,186.95,-0.013105,6.174699
2021-04-07,MSFT,249.90,247.86,0.008230,4.601363,AAPL,127.900,126.21,0.013390,4.692717,^GSPC,4079.95,4073.94,0.001475,1.998026,GME,177.97,184.5,-0.035393,5.956158


In [51]:
# We keep only the percent change column for the stocks we are interested in
msft_aapl_gme_returns = all_together2[['percent_change_msft', 'percent_change_aapl', 'percent_change_gme']]
sp500_returns = all_together2['percent_change_sp500']

In [52]:
sp500_returns

date
2016-04-11         NaN
2016-04-12    0.009662
2016-04-13    0.010040
2016-04-14    0.000173
2016-04-15   -0.000984
                ...   
2021-04-05    0.014438
2021-04-06   -0.000974
2021-04-07    0.001475
2021-04-08    0.004221
2021-04-09    0.007720
Name: percent_change_sp500, Length: 1259, dtype: float64

In [56]:
#We create Dataframes that calculate the excess return for Stocks and Crypto by subtracting the S&P returns to the returns of other stocks
excess1_returns = msft_aapl_gme_returns.sub(sp500_returns, axis = 0)
Crypto_excess = return_crypto.sub(sp500_returns, axis = 0)
excess1_returns.to_csv('../data/excess_stock_returns.csv')
Crypto_excess.to_csv('../data/excess_crypto_returns.csv')

In [59]:
excess1_returns

,percent_change_msft,percent_change_aapl,percent_change_gme
date,,,
2016-04-11,NaN,NaN,NaN
2016-04-12,-0.003402,0.003363,-0.002634
2016-04-13,0.002769,0.004447,0.035822
2016-04-14,0.000008,0.000363,0.005547
2016-04-15,0.006223,-0.019105,0.005408
...,...,...,...
2021-04-05,0.013290,0.009139,-0.037943
2021-04-06,-0.003885,0.003436,-0.012132
2021-04-07,0.006755,0.011915,-0.036868
